In [1]:
import pandas
import numpy
import pygsheets

In [2]:
pandas.__version__

'1.5.0'

# credentials info

In [197]:
credentials_directory = '/Users/baird/Dropbox/_google-api/'
gc = pygsheets.authorize(client_secret=credentials_directory+'client_secret.json')
#spreadsheet = gc.open_by_key('1v38EY_kFFsT17cPqz39j21zRYWeQSSxuDmz565MD2GU') # copy for 2022 report

# sheet 1 (old)

In [220]:
#spreadsheet1 = gc.open_by_key('1MrghwBeCz8Tzgua7CWGg_KoXKVZsV7r0kHMYHYqnNTg') # Aug 18 version
spreadsheet1 = gc.open_by_key('1-dJ_vK3G9pbfTuriMphfRD65ybC99qv3VpjWzMicCzQ') # Aug 19 version, official database

#spreadsheet[1] "Gas Pipelines" tab is the second index
terms_df_orig1 = spreadsheet1.worksheet('title', 'Terminals').get_as_df()

In [222]:
# replace all -- with nans
terms_df_orig1.replace('--', numpy.nan, inplace=True)
# remove oil export terminals
terms_df_orig1 = terms_df_orig1.loc[terms_df_orig1['Fuel']=='LNG']
# remove anything without a wiki page
terms_df_orig1 = terms_df_orig1.loc[terms_df_orig1['Wiki']!='']
# remove N/A statuses
terms_df_orig1 = terms_df_orig1.loc[terms_df_orig1['Status']!='']

terms_df_orig1.set_index('ComboID', inplace=True)

# sheet 2 (new)

In [223]:
spreadsheet2 = gc.open_by_key('1tcS6Wd-Wp-LTDpLzFgJY_RSNDnbyubW3J_9HKIAys4A') # present-day data set
terms_df_orig2 = spreadsheet2.worksheet('title', 'Terminals').get_as_df(start='A2')

In [224]:
# replace all -- with nans
terms_df_orig2.replace('--', numpy.nan, inplace=True)
# remove oil export terminals
terms_df_orig2 = terms_df_orig2.loc[terms_df_orig2['Fuel']=='LNG']
# remove anything without a wiki page
terms_df_orig2 = terms_df_orig2.loc[terms_df_orig2['Wiki']!='']
# remove N/A statuses
terms_df_orig2 = terms_df_orig2.loc[terms_df_orig2['Status']!='']

In [225]:
terms_df_orig2['TerminalName'] = terms_df_orig2.TerminalName.astype(str)+' '+terms_df_orig2.UnitName.astype(str)
terms_df_orig2.set_index('ComboID', inplace=True)

## get shared columns

In [226]:
set(terms_df_orig1.columns)-set(terms_df_orig2.columns)

{'CostUSDPerBcm/y',
 'OtherLanguageTerminalName',
 'OtherNames',
 'ResearcherNotes1',
 'ResearcherNotes2',
 'ResearcherNotes3',
 'ResearcherNotes4',
 'Type2',
 'WriteDown'}

In [227]:
set(terms_df_orig2.columns)-set(terms_df_orig1.columns)

{'CCS',
 'CCSNotes',
 'ConstructionYearMonth',
 'Construction_Sanctioned_July_2022',
 'DelayedStartYear',
 'EuroCrisisTracker',
 'FIDYearMonth',
 'FloatingVesselName',
 'H2',
 'H2Notes',
 'ImpactedByRussiaUkraineInvasion',
 'Offshore',
 'OtherEnglishNames',
 'OtherLanguageName',
 'Owner',
 'PCINumber',
 'ProposalYearMonth',
 'ResearcherNotes',
 'StartYear1YearMonth',
 'Type',
 'UnitName'}

In [228]:
keep_these_columns = list(set(terms_df_orig2.columns).intersection(set(terms_df_orig1.columns)))

## now get shared rows

In [229]:
if 'ComboID' in terms_df_orig2.columns:
    terms_df_orig2.set_index('ComboID', inplace=True)
if 'ComboID' in terms_df_orig1.columns:
    terms_df_orig1.set_index('ComboID', inplace=True)

In [230]:
remove_these_indices = set(terms_df_orig2.index).symmetric_difference(set(terms_df_orig1.index))

In [231]:
new_additions = list(set(terms_df_orig2.index)-set(terms_df_orig1.index))
deleted_since_feb = list(set(terms_df_orig1.index)-set(terms_df_orig2.index))

In [232]:
keep_these_indices = set(terms_df_orig2.index).intersection(set(terms_df_orig1.index))

## if there are diff ComboIDs, pull out only the common ones, THEN compare rest

In [233]:
terms_df_orig1.index.isin(['T020502'])

array([False,  True, False, ..., False, False, False])

In [234]:
common_df1 = terms_df_orig1.loc[terms_df_orig1.index.isin(keep_these_indices),keep_these_columns].sort_index()
common_df2 = terms_df_orig2.loc[terms_df_orig2.index.isin(keep_these_indices),keep_these_columns].sort_index()

common_df1 = common_df1.loc[common_df1.Region=='Europe']
common_df2 = common_df2.loc[common_df2.Region=='Europe']

In [235]:
#index_list = common_df2.compare(common_df1, result_names=('after','before')).index
common_df2.compare(common_df1, result_names=('after','before')).replace(numpy.nan,'')

PCI3        StartYearEarliest        CostEst         PCI5         \
        after before             after before   after before after before   
ComboID                                                                     
T042400                                                                     
T042600                                                                     
T042601                                                                     
T042602                                                                     
T042800                                                                     
...       ...    ...               ...    ...     ...    ...   ...    ...   
T083100                                                                     
T085500                                                                     
T085600                                                                     
T092400           no                                                        
T099200                                                                     

        StartYear1         ...   Latitude            StopYear         \
             after before  ...      after     before    after before   
ComboID                    ...                                         
T042400                    ...                                         
T042600                    ...                                         
T042601                    ...                                         
T042602                    ...                                         
T042800                    ...                                         
...            ...    ...  ...        ...        ...      ...    ...   
T083100                    ...                                         
T085500                    ...  51.264274  51.088313                   
T085600                    ...                                         
T092400                    ...                                         
T099200                    ...                                         

        CostEstUnits         PCI4        ConstructionYear         
               after before after before            after before  
ComboID                                                           
T042400                                                           
T042600                                              1982         
T042601                                                           
T042602                                                           
T042800                                                           
...              ...    ...   ...    ...              ...    ...  
T083100                                                           
T085500                                                           
T085600                                                           
T092400                                                           
T099200                                                           

[112 rows x 74 columns]

In [236]:
diff_df = common_df2.compare(common_df1, result_names=('after','before')).replace(numpy.nan,'')

In [240]:
diff_df[('Region','')] = terms_df_orig2.loc[diff_df.index,'Region']
diff_df[('TerminalName','')] = terms_df_orig2.loc[diff_df.index, 'TerminalName']
diff_df[('Wiki','')] = terms_df_orig2.loc[diff_df.index, 'Wiki']

In [241]:
diff_df.to_excel('differences_before_after_russian_invasion.xlsx')#, index=False)

In [242]:
added_df = terms_df_orig2.loc[new_additions]

In [243]:
added_df.index

Index(['T094301', 'T101200', 'T106801', 'T100600', 'T107100', 'T109001',
       'T107204', 'T108200', 'T060201', 'T022608',
       ...
       'T100301', 'T080904', 'T032001', 'T022610', 'T029801', 'T022000',
       'T060202', 'T080905', 'T108300', 'T108600'],
      dtype='object', name='ComboID', length=128)

In [244]:
terms_df_orig2.columns

Index(['TerminalID', 'ProjectID', 'Wiki', 'TerminalName', 'UnitName', 'Status',
       'Fuel', 'Import/Export', 'Type', 'Country', 'Researcher', 'LastUpdated',
       'OtherEnglishNames', 'Owner', 'Parent', 'ProposalYear',
       'ConstructionYear', 'StartYear1', 'StartYear2', 'StartYear3',
       'DelayedStartYear', 'Delayed', 'DelayType', 'StartYearEarliest',
       'ShelvedYear', 'CancelledYear', 'StopYear',
       'ShelvedCancelledStatusType', 'Capacity', 'CapacityUnits',
       'CapacityInMtpa', 'CapacityInBcm/y', 'Region', 'Location',
       'Prefecture/District', 'State/Province', 'Latitude', 'Longitude',
       'Accuracy', 'Source', 'PowerPlantsSupplied', 'CostEst', 'CostEstUnits',
       'CostEstYear', 'CostEstUSD', 'FID', 'FIDYear', 'ReExport',
       'EuropeTracker', 'PCINumber', 'PCI3', 'PCI4', 'PCI5', 'Offshore',
       'Floating', 'FloatingVesselName', 'Opposition', 'ESJNotes', 'Defeated',
       'OtherLanguageName', 'OtherLanguageWikiPage', 'H2', 'H2Notes', 'CCS',
      

In [246]:
terms_df_orig2.loc[added_df.index,'UnitName']

ComboID
T094301     Phase 2 Expansion
T101200                      
T106801                      
T100600                      
T107100                      
                  ...        
T022000                      
T060202    Revived Project T2
T080905               Phase 5
T108300                      
T108600                      
Name: UnitName, Length: 128, dtype: object

In [247]:
added_df['UnitName'] = terms_df_orig2.loc[added_df.index,'UnitName']

In [248]:
removed_df = terms_df_orig1.loc[deleted_since_feb]

In [249]:
added_df.to_excel('added_after_russian_invasion.xlsx')
removed_df.to_excel('removed_after_russian_invasion.xlsx')

## how did change globally, Europe, Germany?    

### globally

In [256]:
# num of distinct projects
new_comboids = list(set(terms_df_orig2.index)-set(terms_df_orig1.index))
new_projectids = list(set(terms_df_orig2.ProjectID)-set(terms_df_orig1.ProjectID))

print(new_comboids.__len__())
print(new_projectids.__len__())

128
5


# orig1

In [274]:
print(terms_df_orig1.shape)
print(terms_df_orig1.TerminalID.unique().size)
print(terms_df_orig1.index.unique().size)

(1092, 61)
696
1092


In [278]:
print(terms_df_orig1.loc[terms_df_orig1.Region=='Europe'].shape)
print(terms_df_orig1.loc[terms_df_orig1.Region=='Europe'].TerminalID.unique().size)
print(terms_df_orig1.loc[terms_df_orig1.Region=='Europe'].index.unique().size)

(113, 61)
90
113


In [287]:
print(terms_df_orig1.loc[(terms_df_orig1.Region=='Europe')&
                         (terms_df_orig1.Status.isin(['Construction','Proposed']))].shape)
print(terms_df_orig1.loc[(terms_df_orig1.Region=='Europe')&
                         (terms_df_orig1.Status.isin(['Construction','Proposed']))].TerminalID.unique().size)
print(terms_df_orig1.loc[(terms_df_orig1.Region=='Europe')&
                         (terms_df_orig1.Status.isin(['Construction','Proposed']))].index.unique().size)

(33, 61)
28
33


In [283]:
print(terms_df_orig1.loc[terms_df_orig1.Country=='Germany'].shape)
print(terms_df_orig1.loc[terms_df_orig1.Country=='Germany'].TerminalID.unique().size)
print(terms_df_orig1.loc[terms_df_orig1.Country=='Germany'].index.unique().size)

(4, 61)
4
4


In [288]:
print(terms_df_orig1.loc[(terms_df_orig1.Country=='Germany')&
                         (terms_df_orig1.Status.isin(['Construction','Proposed']))].shape)
print(terms_df_orig1.loc[(terms_df_orig1.Country=='Germany')&
                         (terms_df_orig1.Status.isin(['Construction','Proposed']))].TerminalID.unique().size)
print(terms_df_orig1.loc[(terms_df_orig1.Country=='Germany')&
                         (terms_df_orig1.Status.isin(['Construction','Proposed']))].index.unique().size)

(2, 61)
2
2


# orig2

In [279]:
print(terms_df_orig2.shape)
print(terms_df_orig2.TerminalID.unique().size)
print(terms_df_orig2.index.unique().size)

(1182, 73)
735
1182


In [280]:
print(terms_df_orig2.loc[terms_df_orig2.Region=='Europe'].shape)
print(terms_df_orig2.loc[terms_df_orig2.Region=='Europe'].TerminalID.unique().size)
print(terms_df_orig2.loc[terms_df_orig2.Region=='Europe'].index.unique().size)

(131, 73)
105
131


In [285]:
print(terms_df_orig2.loc[(terms_df_orig2.Region=='Europe')&
                         (terms_df_orig2.Status.isin(['Construction','Proposed']))].shape)
print(terms_df_orig2.loc[(terms_df_orig2.Region=='Europe')&
                         (terms_df_orig2.Status.isin(['Construction','Proposed']))].TerminalID.unique().size)
print(terms_df_orig2.loc[(terms_df_orig2.Region=='Europe')&
                         (terms_df_orig2.Status.isin(['Construction','Proposed']))].index.unique().size)

(46, 73)
41
46


In [282]:
print(terms_df_orig2.loc[terms_df_orig2.Country=='Germany'].shape)
print(terms_df_orig2.loc[terms_df_orig2.Country=='Germany'].TerminalID.unique().size)
print(terms_df_orig2.loc[terms_df_orig2.Country=='Germany'].index.unique().size)

(9, 73)
9
9


In [286]:
print(terms_df_orig2.loc[(terms_df_orig2.Country=='Germany')&
                         (terms_df_orig2.Status.isin(['Construction','Proposed']))].shape)
print(terms_df_orig2.loc[(terms_df_orig2.Country=='Germany')&
                         (terms_df_orig2.Status.isin(['Construction','Proposed']))].TerminalID.unique().size)
print(terms_df_orig2.loc[(terms_df_orig2.Country=='Germany')&
                         (terms_df_orig2.Status.isin(['Construction','Proposed']))].index.unique().size)

(8, 73)
8
8


# cost differences